In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

from keras.models import Sequential
from keras.layers import LSTM, Dense

###########################
# 1) Define the SARIMAX–LSTM Pipeline Function
###########################
def sarimax_lstm_pipeline(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    target: str,
    features: list,
    horizon: int,
    sarimax_order=(2, 0, 3),
    sarimax_seasonal_order=(1, 0, 1, 7),
    n_steps_lstm=30,
    epochs=50
):
    """
    Train a SARIMAX model on 'train_data', then train an LSTM on the residuals.
    Forecast for the given 'horizon' steps and return both absolute and normalized error metrics.
    """

    # --------- 1. Fit SARIMAX Model --------- #
    sarimax_model = SARIMAX(
        train_data[target],
        exog=train_data[features],
        order=sarimax_order,
        seasonal_order=sarimax_seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    sarimax_results = sarimax_model.fit(disp=False)

    # Forecast next 'horizon' steps using exogenous variables
    future_exog = test_data[features].iloc[:horizon]
    sarimax_forecast = sarimax_results.get_forecast(steps=horizon, exog=future_exog).predicted_mean

    # --------- 2. Compute Residuals --------- #
    in_sample_preds = sarimax_results.predict(exog=train_data[features])
    train_residuals = train_data[target] - in_sample_preds

    # --------- 3. Train LSTM on Residuals --------- #
    scaler = MinMaxScaler()
    residuals_scaled = scaler.fit_transform(train_residuals.values.reshape(-1, 1))

    def create_sequences(data_array, seq_length):
        X, y = [], []
        for i in range(len(data_array) - seq_length):
            X.append(data_array[i:i+seq_length])
            y.append(data_array[i+seq_length])
        return np.array(X), np.array(y)

    X_train, y_train = create_sequences(residuals_scaled, seq_length=n_steps_lstm)

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps_lstm, 1)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    model.fit(X_train, y_train, epochs=epochs, batch_size=32, verbose=0)

    # --------- 4. Forecast LSTM Residuals --------- #
    recent_seq = residuals_scaled[-n_steps_lstm:].reshape(1, n_steps_lstm, 1)
    lstm_predictions_scaled = []

    for _ in range(horizon):
        pred = model.predict(recent_seq, verbose=0)
        lstm_predictions_scaled.append(pred[0, 0])
        recent_seq = np.append(recent_seq[:, 1:, :], [[[pred[0, 0]]]], axis=1)

    lstm_residual_forecast = scaler.inverse_transform(
        np.array(lstm_predictions_scaled).reshape(-1, 1)
    ).flatten()

    # --------- 5. Combine Forecasts --------- #
    combined_forecast = sarimax_forecast.values + lstm_residual_forecast
    actual_values = test_data[target].iloc[:horizon].values
    mean_actual = np.mean(actual_values)

    # --------- 6. Compute Error Metrics --------- #
    rmse = np.sqrt(mean_squared_error(actual_values, combined_forecast))
    mae = mean_absolute_error(actual_values, combined_forecast)
    mape = np.mean(np.abs((actual_values - combined_forecast) / actual_values)) * 100

    # Normalized versions
    nrmse = rmse / mean_actual * 100
    nmae = mae / mean_actual * 100

    return {
        "Horizon": horizon,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE (%)": mape,
        "nRMSE (%)": nrmse,
        "nMAE (%)": nmae,
        "Forecast": combined_forecast,
        "Actual": actual_values
    }

###########################
# 2) Main Execution Block
###########################
if __name__ == "__main__":
    # Load your dataset
    data = pd.read_csv("temp.csv")

    target = 'loadConsumption'
    feature_sets = {
        "Exogenous Features": [
            'DailyMeanTemperature', 'DailyMeanWindspeed', 'DailyPrecipitation', 'day_of_week',
            'AveragePrice_Electricity_Household', 'AveragePrice_NaturalGas_Household',
            'Economic_Component', 'RenewableEnergy_Component'
        ],
        "Generated Features": [
            'is_weekend', 'is_holiday', 'day_of_year',
            'rolling_7', 'rolling_30', 'rolling_365'
        ],
        "Both Exogenous & Generated": [
            'DailyMeanTemperature', 'DailyMeanWindspeed', 'DailyPrecipitation', 'day_of_week',
            'AveragePrice_Electricity_Household', 'AveragePrice_NaturalGas_Household',
            'Economic_Component', 'RenewableEnergy_Component',
            'is_weekend', 'is_holiday', 'day_of_year',
            'rolling_7', 'rolling_30', 'rolling_365'
        ]
    }

    # Train-test split
    train_data, test_data = train_test_split(data, test_size=0.2, shuffle=False)

    # Define forecasting horizons
    horizons = [1, 7, 30, 90, 180]
    all_results = []

    for feature_set_name, features in feature_sets.items():
        for horizon in horizons:
            print(f"Running {feature_set_name}, horizon={horizon} ...")
            output = sarimax_lstm_pipeline(
                train_data=train_data,
                test_data=test_data,
                target=target,
                features=features,
                horizon=horizon,
                sarimax_order=(2, 0, 3),
                sarimax_seasonal_order=(1, 0, 1, 7),
                n_steps_lstm=30,
                epochs=50
            )
            output["Feature Set"] = feature_set_name
            all_results.append(output)

    # Results to DataFrame
    results_df = pd.DataFrame(all_results)
    print("\n=== Final Evaluation Metrics ===")
    print(results_df[[
        "Feature Set", "Horizon",
        "RMSE", "MAE", "MAPE (%)",
        "nRMSE (%)", "nMAE (%)"
    ]])


Running Exogenous Features, horizon=1 ...
Running Exogenous Features, horizon=7 ...
Running Exogenous Features, horizon=30 ...
Running Exogenous Features, horizon=90 ...
Running Exogenous Features, horizon=180 ...
Running Generated Features, horizon=1 ...


C:\Users\mahmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Running Generated Features, horizon=7 ...


C:\Users\mahmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Running Generated Features, horizon=30 ...


...

Running Both Exogenous & Generated, horizon=90 ...


C:\Users\mahmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Running Both Exogenous & Generated, horizon=180 ...


C:\Users\mahmo\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



=== Final Evaluation Metrics ===
                   Feature Set  Horizon           RMSE           MAE  \
0           Exogenous Features        1    8429.629848   8429.629848   
1           Exogenous Features        7   13424.614004  11060.811144   
2           Exogenous Features       30   19551.862890  17052.630061   
3           Exogenous Features       90   25641.883957  21594.325313   
4           Exogenous Features      180  114932.469705  81983.453077   
5           Generated Features        1   11127.473125  11127.473125   
6           Generated Features        7    8910.082948   7973.451490   
7           Generated Features       30   14483.292499  11190.269769   
8           Generated Features       90   13668.052617  10939.997526   
9           Generated Features      180   12865.235997  10237.150418   
10  Both Exogenous & Generated        1    7929.390051   7929.390051   
11  Both Exogenous & Generated        7    9924.347837   9106.691576   
12  Both Exogenous & Generated

In [2]:
results_df

,Horizon,RMSE,MAE,MAPE (%),nRMSE (%),nMAE (%),Forecast,Actual,Feature Set
0,1,8429.629848,8429.629848,2.953452,2.953452,2.953452,[276986.5351522408],[285416.165],Exogenous Features
1,7,13424.614004,11060.811144,4.259531,4.927208,4.059626,"[279595.8984029732, 290033.8726803588, 293484....","[285416.165, 292664.9975, 284392.5725, 284728....",Exogenous Features
2,30,19551.862890,17052.630061,6.597776,7.201694,6.281132,"[279800.45959071734, 290288.5267025756, 293734...","[285416.165, 292664.9975, 284392.5725, 284728....",Exogenous Features
3,90,25641.883957,21594.325313,8.785954,9.954774,8.383418,"[279026.01147060015, 289481.6852108764, 292892...","[285416.165, 292664.9975, 284392.5725, 284728....",Exogenous Features
4,180,114932.469705,81983.453077,32.616824,44.907781,32.033549,"[275513.82006435015, 285918.4480587768, 289270...","[285416.165, 292664.9975, 284392.5725, 284728....",Exogenous Features
5,1,11127.473125,11127.473125,3.898684,3.898684,3.898684,[274288.6918745369],[285416.165],Generated Features
6,7,8910.082948,7973.451490,2.998031,3.270249,2.926479,"[277140.5903730721, 282082.997681107, 281960.6...","[285416.165, 292664.9975, 284392.5725, 284728....",Generated Features
7,30,14483.292499,11190.269769,4.089087,5.334747,4.121802,"[275566.2328901619, 280456.58874556015, 280286...","[285416.165, 292664.9975, 284392.5725, 284728....",Generated Features
8,90,13668.052617,10939.997526,4.273014,5.306255,4.247161,"[276318.7180891365, 281186.7315678258, 281016....","[285416.165, 292664.9975, 284392.5725, 284728....",Generated Features
9,180,12865.235997,10237.150418,4.005382,5.026858,3.999981,"[275710.2306928963, 280561.3805546422, 280366....","[285416.165, 292664.9975, 284392.5725, 284728....",Generated Features
